# Create Water Level Elevation Map

This notebook creates an elevation map by overlaying water surface polygons from different time steps, colored by their corresponding gage height (elevation).

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from matplotlib.colors import Normalize
from matplotlib import cm
from PIL import Image
import cv2
from skimage import measure

## Configuration

In [ ]:
# Camera and directory configuration
camera_id = 'WI_East_Branch_Pecatonica_River_near_Blanchardville_Bullet'
images_dir = f'{camera_id}/images'
masks_dir = f'{camera_id}/masks'
csv_path = f'{camera_id}/images_and_data.csv'

# Output directory for elevation map
output_dir = f'{camera_id}/elevation_maps'
os.makedirs(output_dir, exist_ok=True)

print(f"Loading data from: {csv_path}")
print(f"Output will be saved to: {output_dir}")

## Load Images and Data

In [ ]:
# Load the CSV with images, masks, and gage height data
df = pd.read_csv(csv_path)

print(f"Loaded {len(df)} records")

# Filter out bad quality masks if quality_flag column exists
if 'quality_flag' in df.columns:
    n_bad = (df['quality_flag'] == 'bad').sum()
    if n_bad > 0:
        print(f"\nFound {n_bad} masks marked as 'bad' - excluding from analysis")
        bad_indices = df[df['quality_flag'] == 'bad'].index.tolist()
        print(f"Excluded indices: {bad_indices}")
        df = df[df['quality_flag'] != 'bad'].reset_index(drop=True)
        print(f"Remaining records after filtering: {len(df)}")
    else:
        print(f"\nAll masks marked as 'good' - no filtering applied")
else:
    print(f"\nNo quality_flag column found - using all masks")
    print(f"(Run notebook 02.5 to review and flag bad masks)")

print(f"\nColumns: {df.columns.tolist()}")
print(f"\nGage height (00065) range: {df['00065'].min():.2f} to {df['00065'].max():.2f} ft")
print(f"\nFirst few rows:")
display(df[['image_names', 'mask_filename', '00065', '00060']].head())

## Helper Functions

In [ ]:
def mask_to_polygons(mask, simplify_tolerance=2.0):
    """
    Convert a binary mask to polygon contours.
    
    Args:
        mask: Binary mask array (boolean or 0/1)
        simplify_tolerance: Tolerance for polygon simplification (higher = simpler)
    
    Returns:
        List of polygons, each as an array of (x, y) coordinates
    """
    # Ensure mask is boolean
    if mask.ndim == 3:
        mask = mask.squeeze()
    mask_bool = mask.astype(bool)
    
    # Find contours using OpenCV
    mask_uint8 = (mask_bool * 255).astype(np.uint8)
    contours, hierarchy = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    polygons = []
    for contour in contours:
        # Simplify contour to reduce number of points
        epsilon = simplify_tolerance
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        # Convert to (x, y) coordinates
        if len(approx) >= 3:  # Need at least 3 points for a polygon
            poly = approx.squeeze()
            if poly.ndim == 2:  # Valid polygon
                polygons.append(poly)
    
    return polygons


def load_mask(mask_path):
    """
    Load a mask from a .npy file.
    
    Args:
        mask_path: Path to .npy mask file
    
    Returns:
        Binary mask array
    """
    mask = np.load(mask_path)
    if mask.ndim == 3:
        mask = mask.squeeze()
    return mask.astype(bool)

## Extract Polygons from All Masks

Convert each water mask to polygon(s) and associate with elevation data.

In [ ]:
# Store all polygons with their elevations
polygon_data = []

print("Extracting polygons from masks...")
for idx, row in df.iterrows():
    mask_filename = row['mask_filename']
    mask_path = os.path.join(masks_dir, mask_filename)
    
    if not os.path.exists(mask_path):
        print(f"Warning: Mask not found: {mask_path}")
        continue
    
    # Load mask and convert to polygons
    mask = load_mask(mask_path)
    polygons = mask_to_polygons(mask, simplify_tolerance=2.0)
    
    # Get elevation (gage height) for this mask
    elevation = row['00065']
    discharge = row['00060']
    image_name = row['image_names']
    
    # Store each polygon with its metadata
    for poly in polygons:
        polygon_data.append({
            'polygon': poly,
            'elevation': elevation,
            'discharge': discharge,
            'image_name': image_name,
            'mask_filename': mask_filename
        })

print(f"\nExtracted {len(polygon_data)} polygons from {len(df)} masks")
print(f"Elevation range: {min(p['elevation'] for p in polygon_data):.2f} to {max(p['elevation'] for p in polygon_data):.2f} ft")

## Create Elevation Map Visualization

Overlay all water surface polygons on a single plot, colored by their gage height.

In [ ]:
# Load a reference image to get dimensions and for background
reference_image_path = os.path.join(images_dir, df.iloc[0]['image_names'])
reference_image = Image.open(reference_image_path)
img_width, img_height = reference_image.size

print(f"Reference image size: {img_width} x {img_height}")
print(f"Using image: {df.iloc[0]['image_names']}")

In [ ]:
# Create the elevation map figure
fig, ax = plt.subplots(figsize=(16, 12), dpi=150)

# Show the reference image as background (optional - can be removed for cleaner view)
ax.imshow(reference_image, alpha=0.3)

# Prepare colormap
elevations = [p['elevation'] for p in polygon_data]
min_elevation = min(elevations)
max_elevation = max(elevations)

norm = Normalize(vmin=min_elevation, vmax=max_elevation)
cmap = cm.get_cmap('viridis')  # Can use 'viridis', 'plasma', 'coolwarm', 'RdYlBu_r', etc.

# Plot each polygon
for poly_data in polygon_data:
    poly = poly_data['polygon']
    elevation = poly_data['elevation']
    
    # Get color for this elevation
    color = cmap(norm(elevation))
    
    # Draw filled polygon with some transparency
    ax.fill(poly[:, 0], poly[:, 1], color=color, alpha=0.4, edgecolor=color, linewidth=1)

# Add colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Gage Height (ft)', fontsize=12, weight='bold')

# Set labels and title
ax.set_xlabel('Image X Coordinate (pixels)', fontsize=11)
ax.set_ylabel('Image Y Coordinate (pixels)', fontsize=11)
ax.set_title(f'Water Surface Elevation Map\n{camera_id}\nGage Height Range: {min_elevation:.2f} - {max_elevation:.2f} ft',
             fontsize=14, weight='bold', pad=20)

# Set axis limits
ax.set_xlim(0, img_width)
ax.set_ylim(img_height, 0)  # Invert y-axis to match image coordinates

plt.tight_layout()

# Save the figure
output_path = os.path.join(output_dir, 'elevation_map_with_background.png')
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"\nSaved elevation map to: {output_path}")

plt.show()

## Create Clean Elevation Map (No Background Image)

In [ ]:
# Create elevation map without background image for cleaner visualization
fig, ax = plt.subplots(figsize=(16, 12), dpi=150)

# White background
ax.set_facecolor('white')

# Plot each polygon
for poly_data in polygon_data:
    poly = poly_data['polygon']
    elevation = poly_data['elevation']
    
    # Get color for this elevation
    color = cmap(norm(elevation))
    
    # Draw filled polygon
    ax.fill(poly[:, 0], poly[:, 1], color=color, alpha=0.6, edgecolor=color, linewidth=1.5)

# Add colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Gage Height (ft)', fontsize=12, weight='bold')

# Set labels and title
ax.set_xlabel('Image X Coordinate (pixels)', fontsize=11)
ax.set_ylabel('Image Y Coordinate (pixels)', fontsize=11)
ax.set_title(f'Water Surface Elevation Map (Clean)\n{camera_id}\nGage Height Range: {min_elevation:.2f} - {max_elevation:.2f} ft',
             fontsize=14, weight='bold', pad=20)

# Set axis limits
ax.set_xlim(0, img_width)
ax.set_ylim(img_height, 0)  # Invert y-axis to match image coordinates
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

plt.tight_layout()

# Save the figure
output_path = os.path.join(output_dir, 'elevation_map_clean.png')
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"\nSaved clean elevation map to: {output_path}")

plt.show()

## Elevation Statistics and Analysis

In [ ]:
# Calculate statistics
elevation_stats = pd.DataFrame([
    {'Metric': 'Minimum Gage Height', 'Value': f"{min(elevations):.2f} ft"},
    {'Metric': 'Maximum Gage Height', 'Value': f"{max(elevations):.2f} ft"},
    {'Metric': 'Elevation Range', 'Value': f"{max(elevations) - min(elevations):.2f} ft"},
    {'Metric': 'Mean Gage Height', 'Value': f"{np.mean(elevations):.2f} ft"},
    {'Metric': 'Median Gage Height', 'Value': f"{np.median(elevations):.2f} ft"},
    {'Metric': 'Number of Time Steps', 'Value': len(df)},
    {'Metric': 'Total Polygons', 'Value': len(polygon_data)},
])

print("\n" + "="*60)
print("ELEVATION MAP STATISTICS")
print("="*60)
display(elevation_stats)

# Create histogram of elevations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of gage heights
ax1.hist(elevations, bins=30, color='steelblue', edgecolor='black', alpha=0.7)
ax1.set_xlabel('Gage Height (ft)', fontsize=11)
ax1.set_ylabel('Number of Observations', fontsize=11)
ax1.set_title('Distribution of Gage Heights', fontsize=12, weight='bold')
ax1.grid(True, alpha=0.3)

# Time series of gage height
df['image_times'] = pd.to_datetime(df['image_times'])
ax2.plot(df['image_times'], df['00065'], marker='o', linestyle='-', linewidth=2, markersize=4, color='steelblue')
ax2.set_xlabel('Time', fontsize=11)
ax2.set_ylabel('Gage Height (ft)', fontsize=11)
ax2.set_title('Gage Height Time Series', fontsize=12, weight='bold')
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()

# Save the figure
stats_path = os.path.join(output_dir, 'elevation_statistics.png')
plt.savefig(stats_path, dpi=150, bbox_inches='tight')
print(f"\nSaved statistics plots to: {stats_path}")

plt.show()

## Export Polygon Data

Save the polygon data for further analysis or use in other applications.

In [ ]:
# Create a summary DataFrame
export_data = []
for i, poly_data in enumerate(polygon_data):
    export_data.append({
        'polygon_id': i,
        'elevation_ft': poly_data['elevation'],
        'discharge_cfs': poly_data['discharge'],
        'image_name': poly_data['image_name'],
        'mask_filename': poly_data['mask_filename'],
        'num_vertices': len(poly_data['polygon'])
    })

export_df = pd.DataFrame(export_data)
export_csv_path = os.path.join(output_dir, 'polygon_elevation_data.csv')
export_df.to_csv(export_csv_path, index=False)

print(f"\nExported polygon data to: {export_csv_path}")
print(f"\nFirst few rows:")
display(export_df.head(10))

## Summary

In [ ]:
print("="*70)
print("ELEVATION MAP CREATION COMPLETE")
print("="*70)
print(f"Camera: {camera_id}")
print(f"Time steps processed: {len(df)}")
print(f"Polygons extracted: {len(polygon_data)}")
print(f"Elevation range: {min_elevation:.2f} - {max_elevation:.2f} ft")
print(f"\nOutput files saved to: {output_dir}")
print(f"  - elevation_map_with_background.png")
print(f"  - elevation_map_clean.png")
print(f"  - elevation_statistics.png")
print(f"  - polygon_elevation_data.csv")
print("="*70)
print("\nThe elevation map shows water surfaces at different gage heights,")
print("with colors representing the water level elevation at each time step.")
print("="*70)